# Using pymrio without a parser (small IO example)

Pymrio provides parsing function to load existing MRIO databases. However, it is also possible to assign data directly to the attributes of an IOSystem instance.

This tutorial exemplify this functionality. The tables used here are taken from *Miller and Blair (2009)*: Miller, Ronald E, and Peter D Blair. Input-Output Analysis: Foundations and Extensions. Cambridge (England); New York: Cambridge University Press, 2009. ISBN: 978-0-521-51713-3


## Preperation

### Import pymrio

First import the pymrio module and other packages needed:

In [1]:
import pymrio

import pandas as pd
import numpy as np

### Get external IO table

For this example we use the IO table given in *Miller and Blair (2009)*: Table 2.3 (on page 22 in the 2009 edition).

This table contains an interindustry trade flow matrix, final demand columns for household demand and exports and a value added row. The latter we consider as an extensions (factor inputs). To assign these values to the IOSystem attributes, the tables must be pandas
<a href="http://pandas.pydata.org/pandas-docs/dev/dsintro.html#dataframe">DataFrames</a> with <a href="http://pandas.pydata.org/pandas-docs/dev/indexing.html#hierarchical-indexing-multiindex">multiindex</a> for columns and index.

First we set up the Z matrix by defining the index of rows and columns. The example IO tables contains only domestic tables, but since pymrio was designed with multi regions IO tables in mind, also a region index is needed.

In [2]:
_sectors = ['sector1', 'sector2']
_regions = ['reg1']
_Z_multiindex = pd.MultiIndex.from_product(
                [_regions, _sectors], names = [u'region', u'sector'])

Next we setup the total Z matrix. Here we just put in the name the values manually. However, pandas provides several possibility to ease the <a href="http://pandas.pydata.org/pandas-docs/stable/io.html">data input</a>.

In [3]:
Z = pd.DataFrame(
    data = np.array([
            [150,500],
            [200,100]]),
    index = _Z_multiindex,
    columns = _Z_multiindex
    )

In [4]:
Z

region            reg1        
sector         sector1 sector2
region sector                 
reg1   sector1     150     500
       sector2     200     100

Final demand is treated in the same way:

In [5]:
_categories = ['final demand']
_fd_multiindex = pd.MultiIndex.from_product(
                 [_regions, _categories], names = [u'region', u'category'])

In [6]:
Y = pd.DataFrame(
    data=np.array([[350], [1700]]),
    index = _Z_multiindex,
    columns = _fd_multiindex)


In [7]:
Y

region                 reg1
category       final demand
region sector              
reg1   sector1          350
       sector2         1700

Factor inputs are given as 'Payment sectors' in the table:

In [8]:
F = pd.DataFrame(
    data = np.array([[650, 1400]]),
    index = ['Payments_sectors'],
    columns = _Z_multiindex)


In [9]:
F

region              reg1        
sector           sector1 sector2
Payments_sectors     650    1400

## Include tables in the IOSystem object

In the next step, an empty instance of an IOSYstem has to be set up.

In [10]:
io = pymrio.IOSystem()

Now we can add the tables to the IOSystem instance:

In [11]:
io.Z = Z
io.Y = Y

Extension are defined as objects within the IOSystem. The Extension instance can be instanced independently (the parameter 'name' is required):

In [12]:
factor_input = pymrio.Extension(name = 'Factor Input', F=F)

In [13]:
io.factor_input = factor_input

For consistency and plotting we can add a DataFrame containg the units per row:

In [14]:
io.factor_input.unit = pd.DataFrame(data = ['USD'], index = F.index, columns = ['unit'])

We can check whats in the system:

In [15]:
str(io)

'IO System with parameters: Z, Y, meta, factor_input'

At this point we have everything to calculate the full IO system.

## Calculate the missing parts

In [16]:
io.calc_all()

This gives, among others, the A and L matrix which we can compare with the tables given in *Miller and Blair (2009)* (Table 2.4 and L given on the next page afterwards):

In [17]:
io.A

region            reg1        
sector         sector1 sector2
region sector                 
reg1   sector1    0.15    0.25
       sector2    0.20    0.05

In [18]:
io.L

region              reg1          
sector           sector1   sector2
region sector                     
reg1   sector1  1.254125  0.330033
       sector2  0.264026  1.122112

## Update to system for a new final demand

The example in *Miller and Blair (2009)* goes on with using the L matrix to calculate the new industry output x for a changing finald demand Y. This step can easly reproduced with the pymrio module.

To do so we first have to set up the new final demand:

In [19]:
Ynew = Y.copy()
Ynew[('reg1','final_demand')] = np.array([[600],
                                          [1500]])

We copy the original IOSystem:

In [20]:
io_new_fd = io.copy()

To calculate for the new final demand we have to remove everything from the system except for the coefficients (A,L,S,M)

In [21]:
io_new_fd.reset_all_to_coefficients()

Now we can assign the new final demand and recalculate the system:

In [22]:
io_new_fd.Y = Ynew

In [23]:
io_new_fd.calc_all()

The new x equalls the xnew values given in *Miller and Blair (2009)* at formula 2.13:

In [24]:
io_new_fd.x

indout
region sector              
reg1   sector1  2247.524752
       sector2  3841.584158

As for all IO System, we can have a look at the modification history:

In [25]:
io_new_fd.meta

Description: Metadata for pymrio
MRIO Name: IO_copy
System: None
Version: None
File: None
History:
20210224 10:42:06 - MODIFICATION -  Calculating accounts for extension factor_input
20210224 10:42:06 - MODIFICATION -  Flow matrix Z calculated
20210224 10:42:06 - MODIFICATION -  Industry Output x calculated
20210224 10:42:06 - MODIFICATION -  Reset full system to coefficients
20210224 10:42:06 - NOTE -  IOSystem copy IO_copy based on IO
20210224 10:42:06 - MODIFICATION -  Calculating accounts for extension factor_input
20210224 10:42:06 - MODIFICATION -  Leontief matrix L calculated
20210224 10:42:06 - MODIFICATION -  Coefficient matrix A calculated
20210224 10:42:06 - MODIFICATION -  Industry output x calculated